In [1]:
from skbio import TreeNode
import gzip
from collections import Counter, deque
import time
import numpy as np
import pandas as pd
def extract_consenus(file):
    f = open(file)
    g_id = f.readline()[1:].strip()
    seq = ""
    lines = iter(f)
    for line in lines:
        if line[0] == '>':
            yield (g_id, seq)
            g_id = line[1:].strip()
            seq = "" 
        else:
            seq += line.strip()

def extract_sample_read_counts(file):
    f = gzip.open(file, 'rt')
    s_count = Counter()
    i = 0
    for idx, line in enumerate(f):
        if idx % 4 == 1:
            s_count[line.strip()] += 1
    
    return s_count

In [2]:
consenus_genome = deque(
    extract_consenus("2021-08-23_21-01-26-all_stringent_only.fas"))
genomes = [genome for genome, _ in consenus_genome]
sequences = [sequence for _, sequence in consenus_genome]
d = {'genome': genomes, 'sequences': sequences}
consensus_genome_df = pd.DataFrame(data=d)
consensus_genome_df

,genome,sequences
0,Consensus_SEARCH-10143__EXC_REP_001378LIB__G12...,NNTAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
1,Consensus_SEARCH-10386__EXC_REP_001533LIB__A01...,NNTAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
2,Consensus_SEARCH-12610__EXC_REP_001533LIB__C10...,NNTAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
3,Consensus_SEARCH-12615__EXC_REP_001533LIB__E10...,NNTAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
4,Consensus_SEARCH-12616__EXC_REP_001533LIB__G10...,NNTAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
...,...,...
16565,Consensus_SEARCH-46863__E0001008__D15__210820_...,NNTAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
16566,Consensus_SEARCH-46864__E0001008__E15__210820_...,NNNAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
16567,Consensus_SEARCH-46865__E0001008__F15__210820_...,NNNAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
16568,Consensus_SEARCH-46866__E0001008__G15__210820_...,NNTAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...


In [3]:
# sample 38879
r1 = [(count, read) for read, count in extract_sample_read_counts(
        "SEARCH-38879__E0001197__P24__210924_A01535_0019_BHT7MHDSX2__S736_L004_R1_001.fastq.gz").items() 
          if count > 1]
r2 = [(count, seq) for seq, count in extract_sample_read_counts(
        "SEARCH-38879__E0001197__P24__210924_A01535_0019_BHT7MHDSX2__S736_L004_R2_001.fastq.gz").items()
           if count > 1]

In [14]:
t1 = time.clock()
matched_seq = deque()
reads = r1[0][1]
for count, read in r1[:1] + r2:
    reads += '|' + read
found = consensus_genome_df.loc[consensus_genome_df.sequences.str.contains(reads)]
t2 = time.clock()
print(t2-t1)

80.235287


In [15]:
found

,genome,sequences
0,Consensus_SEARCH-10143__EXC_REP_001378LIB__G12...,NNTAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
1,Consensus_SEARCH-10386__EXC_REP_001533LIB__A01...,NNTAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
2,Consensus_SEARCH-12610__EXC_REP_001533LIB__C10...,NNTAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
3,Consensus_SEARCH-12615__EXC_REP_001533LIB__E10...,NNTAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
4,Consensus_SEARCH-12616__EXC_REP_001533LIB__G10...,NNTAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
...,...,...
16565,Consensus_SEARCH-46863__E0001008__D15__210820_...,NNTAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
16566,Consensus_SEARCH-46864__E0001008__E15__210820_...,NNNAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
16567,Consensus_SEARCH-46865__E0001008__F15__210820_...,NNNAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
16568,Consensus_SEARCH-46866__E0001008__G15__210820_...,NNTAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...


In [41]:
t1 = time.clock()
matched_seq = deque()
reads = r1[0][1]
for count, read in r1[:1] + r2:
    if count > 10000:
        reads += '|' + read
found = consensus_genome_df.loc[consensus_genome_df.sequences.str.contains(reads)]
t2 = time.clock()
print(t2-t1)

15.282468999999992


In [42]:
found

,genome,sequences
0,Consensus_SEARCH-10143__EXC_REP_001378LIB__G12...,NNTAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
1,Consensus_SEARCH-10386__EXC_REP_001533LIB__A01...,NNTAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
2,Consensus_SEARCH-12610__EXC_REP_001533LIB__C10...,NNTAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
3,Consensus_SEARCH-12615__EXC_REP_001533LIB__E10...,NNTAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
4,Consensus_SEARCH-12616__EXC_REP_001533LIB__G10...,NNTAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
...,...,...
16565,Consensus_SEARCH-46863__E0001008__D15__210820_...,NNTAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
16566,Consensus_SEARCH-46864__E0001008__E15__210820_...,NNNAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
16567,Consensus_SEARCH-46865__E0001008__F15__210820_...,NNNAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
16568,Consensus_SEARCH-46866__E0001008__G15__210820_...,NNTAAAGGTTTATACCCTTCCCAGGTAACAAACCAACCAACTTTCG...
